In [17]:
import sys
import pandas as pd
import numpy as np
import xarray as xr
sys.path.insert(0, '../..')
from config import DATA_DIR
unique_years = np.arange(2003, 2022)

yearly_data = []

# Load and preprocess the data for each year
for year in unique_years:
    file = DATA_DIR / f"hazard_input_data/isimip3a/flood_fraction/cama-flood_ensemble_median_20crv3-era5_obsclim_histsoc_default_fldfrc_15arcmin_global_month-max_nopros_{year}.nc"
    yearly_hazard = xr.open_dataset(file)
    yearly_hazard = yearly_hazard.sel(time=str(year))
    yearly_hazard = yearly_hazard.rename({"lat": "latitude", "lon": "longitude"})
    
    # Select the flood data variable
    flood_data = yearly_hazard['fldfrc']  
    
    # Append the data to the list
    yearly_data.append(flood_data)

# Concatenate all yearly data along a new time dimension
combined_data = xr.concat(yearly_data, dim="time").resample(time='1Y').max()

# Calculate the median across the time dimension (years)
median_flood = combined_data.median(dim="time")

In [16]:
combined_data

<xarray.DataArray 'fldfrc' (time: 228, latitude: 720, longitude: 1440)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2003-01-16T12:00:00 ... 2021-12-16
  * longitude  (longitude) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * latitude   (latitude) float32 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
Attributes:
    long_name:  flooded fraction
    units:      0-1

In [19]:
# Iterate over each year's data to set values below the median to 0
for i, year in enumerate(unique_years):
    yearly_data_corrected = yearly_data[i].where(yearly_data[i] > median_flood, 0)

    # Save the modified yearly data to a new NetCDF file
    output_file = DATA_DIR / f"hazard_input_data/isimip3a/flood_fraction/cama-flood_ensemble_median_20crv3-era5_obsclim_histsoc_default_fldfrc_15arcmin_global_month-max_2y_{year}.nc"
    yearly_data_corrected.to_netcdf(output_file)


In [23]:
yearly_data[10].sel(latitude=39, longitude=78, method='nearest')

<xarray.DataArray 'fldfrc' (time: 12)>
array([0.      , 0.008072, 0.01027 , 0.009073, 0.012012, 0.026665, 0.021618,
       0.021982, 0.02027 , 0.01247 , 0.      , 0.      ], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2013-01-16T12:00:00 ... 2013-12-16
    longitude  float32 78.12
    latitude   float32 39.12
Attributes:
    long_name:  flooded fraction
    units:      0-1